In [1]:
import numpy as np 
import pandas as pd 
from catboost import * 
from PIL import Image 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import os
import torch
import torch.nn as nn
import optuna

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

/opt/ml/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def change_age(x):
	if x < 10: return 0
	elif 10 <= x < 20: return 1
	elif 20 <= x < 30: return 2
	elif 30 <= x < 40: return 3
	elif 40 <= x < 50: return 4
	elif 50 <= x < 60: return 5
	elif 60 <= x < 70: return 6
	elif 70 <= x < 100: return 7
	else: return 8
 
def change_count(x):
    if x < 2: return 0 
    elif 2 <= x < 3: return 1
    elif 3 <= x < 5: return 2
    elif 5 <= x < 7: return 3 
    elif 7 <= x < 10: return 4
    elif 10 <= x < 30: return 5 
    elif 30 <= x < 100: return 6
    else: return 7

In [17]:
dpath = 'data/v5/'
users = pd.read_csv(os.path.join(dpath,'users.csv'))
books = pd.read_csv(os.path.join(dpath,'books.csv'))
train = pd.read_csv(os.path.join(dpath,'train_ratings.csv'))
test = pd.read_csv(os.path.join(dpath,'test_ratings.csv'))
submission = pd.read_csv(os.path.join(dpath,'sample_submission.csv'))

In [18]:
users['age'] = users['age'].apply(change_age)
users['rating_count'] = users['rating_count'].apply(change_count)

In [19]:
books['years'] = books['year_of_publication'].copy()
books['years'][books['year_of_publication'] < 1970] = 1970
books['years'][(books['year_of_publication'] < 1980) * (books['year_of_publication'] >= 1970)] = 1980
books['years'][(books['year_of_publication'] < 1990) * (books['year_of_publication'] >= 1980)] = 1990
books['years'][(books['year_of_publication'] < 2000) * (books['year_of_publication'] >= 1990)] = 2000
books['years'][(books['year_of_publication'] >= 2000)] = 2020
books['years'] = books['years'].astype('str')
#books['years'] = books['years'].astype('int')
books.drop(['year_of_publication', 'category'], axis = 1, inplace = True)


/tmp/ipykernel_59588/2385907523.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books['years'][books['year_of_publication'] < 1970] = 1970
/tmp/ipykernel_59588/2385907523.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books['years'][(books['year_of_publication'] < 1980) * (books['year_of_publication'] >= 1970)] = 1980
/tmp/ipykernel_59588/2385907523.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books['years'][(books['yea

In [20]:
ids = pd.concat([train['user_id'], submission['user_id']]).unique()
isbns = pd.concat([train['isbn'], submission['isbn']]).unique()

idx2user = {idx:id for idx, id in enumerate(ids)}
idx2isbn = {idx:isbn for idx, isbn in enumerate(isbns)}

user2idx = {id:idx for idx, id in idx2user.items()}
isbn2idx = {isbn:idx for idx, isbn in idx2isbn.items()}

users_ = users.copy()
books_ = books.copy()
books_ = books_.drop(['img_url','img_path'],axis=1)

In [21]:
train = pd.merge(train, users_, on='user_id', how='left')
submission = pd.merge(submission, users_, on='user_id', how='left')
test = pd.merge(test, users_, on='user_id', how='left')

train = pd.merge(train, books_, on='isbn', how='left')
submission = pd.merge(submission, books_, on='isbn', how='left')
test = pd.merge(test, books_, on='isbn', how='left')

train['user_id'] = train['user_id'].map(user2idx)
submission['user_id'] = submission['user_id'].map(user2idx)
test['user_id'] = test['user_id'].map(user2idx)

train['isbn'] = train['isbn'].map(isbn2idx)
submission['isbn'] = submission['isbn'].map(isbn2idx)
test['isbn'] = test['isbn'].map(isbn2idx)

In [7]:
train['year_of_publication'] = train['year_of_publication'].astype(int)
submission['year_of_publication'] = submission['year_of_publication'].astype(int)
test['year_of_publication'] = test['year_of_publication'].astype(int)

In [22]:
train = train.fillna('-1')
submission = submission.fillna('-1')
test = test.fillna('-1')

sub_rating = submission['rating']
submission = submission.drop(columns='rating')
submission['rating'] = sub_rating


In [ ]:
train = train.drop(columns=['category'])
test = test.drop(columns=['category'])
submission = submission.drop(columns=['category'])

In [27]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['rating']):
    folds.append((train_idx,valid_idx))


In [29]:
import random

def rmse(real: list, predict: list) -> float:
    pred = np.array(predict)
    return np.sqrt(np.mean((real-pred) ** 2))

SEED = 42
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)

In [31]:
cat_models={}

cat_features = list(range(1, 10))

def objective(trial):
    param = {
        "random_state":42,
        "objective" : "RMSE",
        "cat_features" : list(train.drop(['rating'],axis = 1).columns),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
        "max_depth":trial.suggest_int("max_depth", 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
    #   "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0), 이거 때메 GPU 안돌아감
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
    }

    model = CatBoostRegressor(**param, task_type = 'GPU', devices = '0')

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    cat_pred = model.predict(X_valid)
    log_score = rmse(y_valid, cat_pred)

    return log_score

In [32]:
for fold in range(0,10):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train = train.drop(['rating'],axis = 1).iloc[train_idx]
    X_valid = train.drop(['rating'],axis = 1).iloc[valid_idx]
    y_train = train['rating'].iloc[train_idx]
    y_valid = train['rating'].iloc[valid_idx]

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(
        study_name = 'cat_parameter_opt',
        direction = 'minimize',
        sampler = sampler,
    )
    study.optimize(objective, n_trials=10)


    model = CatBoostRegressor(**study.best_params, task_type = 'GPU', 
                              devices = '0', random_state = SEED, objective = 'RMSE', 
                              cat_features = list(train.drop(['rating'],axis = 1).columns))
    model.fit(X_train, y_train)
                
    pred = model.predict(test.drop(['rating'],axis = 1))
    test[f'pred_{fold}'] = pred
    print(f'================================================================================\n\n')

[I 2023-04-19 02:04:24,133] A new study created in memory with name: cat_parameter_opt


====================================1============================================


/tmp/ipykernel_59588/2367454807.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.5),
/tmp/ipykernel_59588/2367454807.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),


0:	learn: 2.4327539	test: 2.4326590	best: 2.4326590 (0)	total: 17.1ms	remaining: 2m 9s
100:	learn: 2.4278252	test: 2.4250672	best: 2.4250672 (100)	total: 1.52s	remaining: 1m 52s
200:	learn: 2.4278245	test: 2.4250362	best: 2.4250362 (194)	total: 3.01s	remaining: 1m 50s
bestTest = 2.425035829
bestIteration = 226
Shrink model to first 227 iterations.


[I 2023-04-19 02:04:31,792] Trial 0 finished with value: 2.425036035871186 and parameters: {'learning_rate': 0.043284502212938815, 'bagging_temperature': 63.512210106407046, 'n_estimators': 7588, 'max_depth': 11, 'random_strength': 15, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 2.425036035871186.
/tmp/ipykernel_59588/2367454807.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.5),
/tmp/ipykernel_59588/2367454807.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_temperature' :trial

...

0:	learn: 2.3672644	test: 2.3551897	best: 2.3551897 (0)	total: 15.1ms	remaining: 60s
100:	learn: 2.2000119	test: 2.1586932	best: 2.1586790 (95)	total: 1.99s	remaining: 1m 16s
200:	learn: 2.1938476	test: 2.1557357	best: 2.1557357 (200)	total: 3.83s	remaining: 1m 12s
300:	learn: 2.1903120	test: 2.1537729	best: 2.1537369 (296)	total: 5.71s	remaining: 1m 9s
400:	learn: 2.1871979	test: 2.1525822	best: 2.1525607 (398)	total: 7.53s	remaining: 1m 7s
500:	learn: 2.1846329	test: 2.1515819	best: 2.1515393 (486)	total: 9.4s	remaining: 1m 5s
600:	learn: 2.1822750	test: 2.1508302	best: 2.1508268 (599)	total: 11.2s	remaining: 1m 3s
700:	learn: 2.1800585	test: 2.1505534	best: 2.1505528 (696)	total: 13.1s	remaining: 1m 1s
800:	learn: 2.1774500	test: 2.1503061	best: 2.1501137 (786)	total: 14.9s	remaining: 59.2s
900:	learn: 2.1749748	test: 2.1499141	best: 2.1498752 (891)	total: 16.7s	remaining: 57.2s
1000:	learn: 2.1728373	test: 2.1496848	best: 2.1496085 (994)	total: 18.6s	remaining: 55.2s
1100:	learn: 2

[I 2023-04-19 07:06:27,962] Trial 8 finished with value: 2.144936082215738 and parameters: {'learning_rate': 0.29267581150621286, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 3978, 'max_depth': 4, 'random_strength': 31, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 2 with value: 2.127316198662639.
/tmp/ipykernel_59588/2367454807.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.5),
/tmp/ipykernel_59588/2367454807.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_temperature' :tri

0:	learn: 2.4301204	test: 2.4298873	best: 2.4298873 (0)	total: 44.3ms	remaining: 5m 47s
100:	learn: 2.2705892	test: 2.2683694	best: 2.2683694 (100)	total: 4.56s	remaining: 5m 50s
200:	learn: 2.2266533	test: 2.2319093	best: 2.2319093 (200)	total: 8.92s	remaining: 5m 39s
300:	learn: 2.1986938	test: 2.2138574	best: 2.2138574 (300)	total: 13.3s	remaining: 5m 34s
400:	learn: 2.1785664	test: 2.2047057	best: 2.2047057 (400)	total: 17.8s	remaining: 5m 30s
500:	learn: 2.1620362	test: 2.1987355	best: 2.1987355 (500)	total: 22.4s	remaining: 5m 28s
600:	learn: 2.1470429	test: 2.1942585	best: 2.1942585 (600)	total: 26.9s	remaining: 5m 24s
700:	learn: 2.1335295	test: 2.1908842	best: 2.1908810 (699)	total: 31.3s	remaining: 5m 19s
800:	learn: 2.1205751	test: 2.1883369	best: 2.1883271 (799)	total: 35.7s	remaining: 5m 13s
900:	learn: 2.1083993	test: 2.1856337	best: 2.1856174 (899)	total: 40s	remaining: 5m 8s
1000:	learn: 2.0962139	test: 2.1834745	best: 2.1834745 (1000)	total: 44.3s	remaining: 5m 3s
1100

[I 2023-04-19 07:07:47,977] Trial 9 finished with value: 2.175549795255116 and parameters: {'learning_rate': 0.015965665886173763, 'bagging_temperature': 7.128188058401368, 'n_estimators': 7847, 'max_depth': 11, 'random_strength': 77, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 2 with value: 2.127316198662639.


0:	learn: 2.4074898	total: 33ms	remaining: 1m 59s
1:	learn: 2.3830459	total: 62.3ms	remaining: 1m 52s
2:	learn: 2.3644796	total: 89.4ms	remaining: 1m 47s
3:	learn: 2.3454910	total: 119ms	remaining: 1m 48s
4:	learn: 2.3285451	total: 147ms	remaining: 1m 46s
5:	learn: 2.3131203	total: 178ms	remaining: 1m 47s
6:	learn: 2.3035763	total: 205ms	remaining: 1m 46s
7:	learn: 2.2944192	total: 235ms	remaining: 1m 46s
8:	learn: 2.2849308	total: 265ms	remaining: 1m 46s
9:	learn: 2.2750899	total: 296ms	remaining: 1m 47s
10:	learn: 2.2662000	total: 327ms	remaining: 1m 47s
11:	learn: 2.2588469	total: 357ms	remaining: 1m 47s
12:	learn: 2.2538942	total: 389ms	remaining: 1m 48s
13:	learn: 2.2483605	total: 419ms	remaining: 1m 48s
14:	learn: 2.2431059	total: 451ms	remaining: 1m 48s
15:	learn: 2.2394386	total: 484ms	remaining: 1m 49s
16:	learn: 2.2357240	total: 514ms	remaining: 1m 49s
17:	learn: 2.2332884	total: 543ms	remaining: 1m 48s
18:	learn: 2.2307656	total: 576ms	remaining: 1m 49s
19:	learn: 2.2279026	

In [34]:
test['rating'] = (test['pred_0'] + test['pred_1'] + test['pred_2'] + test['pred_3'] + test['pred_4'] +test['pred_5'] + test['pred_6'] + test['pred_7'] + test['pred_8'] + test['pred_9']) / 10
submitwant = test[['user_id', 'isbn', 'rating']]
#test.to_csv('../submit/CAT_10Fold.csv', index = False)


In [40]:
submitwant

,user_id,isbn,rating
0,13,0,7.299962
1,13426,0,7.453777
2,26761,1,7.972428
3,16495,2,7.630204
4,6225,3,7.532972
...,...,...,...
76694,7728,149565,5.845773
76695,47785,149566,6.252111
76696,4209,149567,6.634338
76697,40779,149568,4.940003


In [37]:
sample_submission = pd.read_csv(os.path.join(dpath,'sample_submission.csv'))
sample_submission

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0
3,157969,0374157065,0
4,67958,0399135782,0
...,...,...,...
76694,278543,1576734218,0
76695,278563,3492223710,0
76696,278633,1896095186,0
76697,278668,8408044079,0


In [41]:
sample_submission['rating'] = submitwant['rating']
sample_submission.to_csv('code/submit/catboost_10fold.csv',index=False)

In [ ]:

submitwant.to_csv('submit/CatBoost.csv', index = False)

In [42]:
DCN = pd.read_csv('code/submit/20230418_122602_DCN.csv') 
FFM = pd.read_csv('code/submit/20230418_123934_FFM.csv')


In [45]:
def cosine_similarity(v1, v2):
    """두 벡터의 코사인 유사도를 계산하는 함수"""
    dot_product = np.dot(v1, v2) # 두 벡터의 내적(dot product) 계산
    norm_v1 = np.linalg.norm(v1) # 첫 번째 벡터의 노름(norm) 계산
    norm_v2 = np.linalg.norm(v2) # 두 번째 벡터의 노름(norm) 계산
    similarity = dot_product / (norm_v1 * norm_v2) # 코사인 유사도 계산
    return similarity

cosine_similarity(np.array(DCN['rating']), np.array(FFM['rating']))

0.9931955781507921

In [47]:
cosine_similarity(np.array(DCN['rating']), np.array(submitwant['rating']))

0.995552267946165

In [44]:
DCN

,user_id,isbn,rating
0,11676,0002005018,6.168999
1,116866,0002005018,6.931113
2,152827,0060973129,7.318938
3,157969,0374157065,6.812364
4,67958,0399135782,7.101904
...,...,...,...
76694,278543,1576734218,6.366093
76695,278563,3492223710,6.653720
76696,278633,1896095186,6.849774
76697,278668,8408044079,5.168164


In [43]:
submitwant

,user_id,isbn,rating
0,13,0,7.299962
1,13426,0,7.453777
2,26761,1,7.972428
3,16495,2,7.630204
4,6225,3,7.532972
...,...,...,...
76694,7728,149565,5.845773
76695,47785,149566,6.252111
76696,4209,149567,6.634338
76697,40779,149568,4.940003


In [10]:
X_train, X_valid, y_train, y_valid =  train_test_split(train.drop(['rating'],axis=1), train['rating'], test_size=0.2, random_state=42,shuffle=True)

In [11]:
def rmse(real: list, predict: list) -> float:
    pred = np.array(predict)
    return np.sqrt(np.mean((real-pred) ** 2))

class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()
        self.eps = 1e-6

    def forward(self, x, y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y)+self.eps)
        return loss

In [12]:
X_train

,user_id,isbn,age,location_city,location_state,location_country,rating_count,book_title,book_author,year_of_publication,publisher,language,summary,category_high
121312,22946,15310,4,arlington,virginia,usa,22,Grendel,John Champlin Gardner,1989,Vintage Books USA,en,The Beowulf story retold from the monster&#39;...,fiction
265089,44452,91405,6,portland,oregon,usa,5,Solomon's Decision,Judith B. Glad,2002,Tokyopop,en,-1,fiction
60236,15913,4800,3,fortcollins,colorado,usa,9,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,Scribner,en,A whimsical adaptation of classic fairy tales ...,humor
111218,1339,13043,2,badaxe,michigan,usa,113,Eyes of Prey,John Sandford,2004,Berkley Publishing Group,en,When a series of gruesome mutilation killings ...,fiction
306001,15663,128985,4,amora,estremadura,estremadura,71,Culture Shock!: South Africa,Dee Rissik,1993,Health Communications,en,-1,fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,2752,14936,3,fpo,ap,usa,174,The Genesis Code,John Case,1998,Ballantine Books,en,"In a suburb of Washington, D.C., Joe Lassiter ...",fiction
259178,8108,86566,3,newalbany,indiana,usa,4,The Marriage Dictionary,Tom Carey,1995,Pub Group West,en,-1,fiction
131932,10534,18168,5,westbloomfield,michigan,usa,8,Billy Straight : A Novel,JONATHAN KELLERMAN,1999,Ballantine Books,en,-1,fiction
146867,40246,22419,2,noisylegrand,seinestdenis,france,1,La Vie est ailleurs,Milan Kundera,1976,Gallimard,fr,"Postface, p. 397-405, écrite en 1978 (Lire Kun...",fiction


In [13]:
X_train.shape[1]

14

In [14]:
cat_features = [0,1,2,3,4,5,7,8,9,10,11,12,13]
model = CatBoostRegressor(iterations=10000, depth=6, learning_rate=0.05, random_seed=42, verbose=50)
model.fit(X_train,
          y_train,
          cat_features = cat_features,
          eval_set = (X_valid,y_valid),
          use_best_model=True,
          early_stopping_rounds=1000)

0:	learn: 2.4177317	test: 2.4139831	best: 2.4139831 (0)	total: 208ms	remaining: 34m 36s
50:	learn: 2.2222666	test: 2.1726223	best: 2.1726223 (50)	total: 5.36s	remaining: 17m 26s
100:	learn: 2.2076133	test: 2.1549479	best: 2.1549479 (100)	total: 15.6s	remaining: 25m 25s
150:	learn: 2.2021557	test: 2.1504152	best: 2.1504152 (150)	total: 26.9s	remaining: 29m 17s
200:	learn: 2.1982525	test: 2.1470924	best: 2.1470924 (200)	total: 38.6s	remaining: 31m 20s
250:	learn: 2.1953100	test: 2.1447304	best: 2.1447304 (250)	total: 50s	remaining: 32m 22s
300:	learn: 2.1928091	test: 2.1431838	best: 2.1431834 (299)	total: 1m 1s	remaining: 33m 15s
350:	learn: 2.1905541	test: 2.1418624	best: 2.1418624 (350)	total: 1m 14s	remaining: 33m 56s
400:	learn: 2.1885477	test: 2.1408626	best: 2.1408589 (399)	total: 1m 25s	remaining: 34m 11s
450:	learn: 2.1871126	test: 2.1402055	best: 2.1402055 (450)	total: 1m 36s	remaining: 34m 12s
500:	learn: 2.1858131	test: 2.1396999	best: 2.1396999 (500)	total: 1m 48s	remaining: 

In [15]:
model.get_params()

{'iterations': 10000,
 'learning_rate': 0.05,
 'depth': 6,
 'loss_function': 'RMSE',
 'random_seed': 42,
 'verbose': 50}

In [17]:
model.get_best_score()

{'learn': {'RMSE': 2.117305368729774},
 'validation': {'RMSE': 2.1325259289008662}}

In [18]:
pred = model.predict(submission.drop(['rating'],axis=1))
len(pred)

76699

In [19]:
sample_submission = pd.read_csv(os.path.join(dpath,'sample_submission.csv'))
len(sample_submission)

76699

In [38]:
pred_submission = submission.copy()
pred_submission['rating'] = pred

In [20]:
sample_submission['rating'] = pred

In [58]:
os.getcwd()

'/opt/ml'

In [21]:
sample_submission.to_csv('code/submit/catboost_v5_submission.csv',index=False)

In [50]:
pred_submission['user_id'].map(idx2user)

0         11676
1        116866
2        152827
3        157969
4         67958
          ...  
76694    278543
76695    278563
76696    278633
76697    278668
76698    278851
Name: user_id, Length: 76699, dtype: int64

In [54]:
pred_submission['isbn'].map(idx2isbn)

0        0002005018
1        0002005018
2        0060973129
3        0374157065
4        0399135782
            ...    
76694    1576734218
76695    3492223710
76696    1896095186
76697    8408044079
76698    0767907566
Name: isbn, Length: 76699, dtype: object

In [70]:
model.feature_importances_

array([6.03397012e+01, 4.42881231e-02, 1.87064969e+00, 2.12270226e+00,
       7.04342400e+00, 1.60583707e+01, 2.03785915e+00, 7.26898546e+00,
       6.49503360e-01, 2.39280077e+00, 1.71715328e-01])

In [69]:
X_train.columns

Index(['user_id', 'isbn', 'location', 'age', 'book_title', 'book_author',
       'year_of_publication', 'publisher', 'language', 'category', 'summary'],
      dtype='object')

In [71]:
pd.DataFrame({'feature':X_train.columns,'importance':model.feature_importances_}).sort_values(by='importance',ascending=False)

,feature,importance
0,user_id,60.339701
5,book_author,16.058371
7,publisher,7.268985
4,book_title,7.043424
9,category,2.392801
3,age,2.122702
6,year_of_publication,2.037859
2,location,1.870650
8,language,0.649503
10,summary,0.171715


In [75]:
len(os.listdir('data/images/'))

149570